
# Wellness 심리 상담 데이터에 대한 KoGPT2 학습

## 1.Google Drive 연동
- 모델 파일과 학습 데이터가 저장 되어있는 구글 드라이브의 디렉토리와 Colab을 연동.  
- 좌측상단 메뉴에서 런타임-> 런타임 유형 변경 -> 하드웨어 가속기 -> GPU 선택 후 저장

### 1.1 GPU 연동 확인

In [1]:
!nvidia-smi

Thu Oct 22 11:16:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 1.2 Google Drive 연동
아래 코드를 실행후 나오는 URL을 클릭하여 나오는 인증 코드 입력

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Colab 디렉토리 아래 dialogLM 경로 확인**




In [3]:
!ls drive/'My Drive'/'WEB_Ask_06devbros'/'ai'/'chatbot'

checkpoint  Dockerfile	exp.ipynb   model	requirements.txt
data	    docs	main.ipynb  preprocess	service
dataloader  example	main.py     README.md	train


**필요 패키지 설치**

In [4]:
!pip install -r drive/'My Drive'/'WEB_Ask_06devbros'/'ai'/'chatbot'/requirements.txt

     |████████████████████████████████| 778kB 12.7MB/s 
     |████████████████████████████████| 2.9MB 44.3MB/s 
     |████████████████████████████████| 890kB 52.1MB/s 
     |████████████████████████████████| 1.1MB 58.4MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251535 sha256=cbd3ce6cce709111f1b8e423fb584e133d5eec9db839735b5e82caf8c3ad7201
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=a56d78cfb78e1821e62f3018b350b754ab01ff0c88994bce8aa9b6f3c1b3cb84
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built kss sacremoses
ERROR: transformers 3.0.2 has requirement tokenizers==0.8.1.rc1, but you'll have tokenizers 0.9.2 which is incompatible.


In [5]:
!pip install --upgrade tokenizers==0.8.1.rc1

     |████████████████████████████████| 3.0MB 8.3MB/s 
  Found existing installation: tokenizers 0.9.2
    Uninstalling tokenizers-0.9.2:
      Successfully uninstalled tokenizers-0.9.2


## KoGPT-2 Training

**Path 추가**

In [6]:
import sys
sys.path.append('/content/drive/My Drive/WEB_Ask_06devbros/ai/chatbot')

In [7]:
print(sys.path)

['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/My Drive/WEB_Ask_06devbros/ai/chatbot']


### 2.1 import package

In [8]:
import os
import numpy as np
from tqdm import tqdm

import torch
from torch.utils.data import dataloader
from dataloader.wellness import WellnessAutoRegressiveDataset
from model.kogpt2 import DialogKoGPT2

In [9]:
torch.cuda.is_available()

True

In [10]:
!nvidia-smi

Thu Oct 22 11:18:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### KoGPT2 Training for Wellness dataset

In [11]:
root_path='/content/drive/My Drive/WEB_Ask_06devbros/ai/chatbot'
data_path = f"{root_path}/data/chatbot_wellness_dialog_for_autoregressive.txt"
checkpoint_path =f"{root_path}/checkpoint"
load_ckpt_path = f"{checkpoint_path}/kogpt2-wellnesee-auto-regressive-20201022.pth"
save_ckpt_path = f"{checkpoint_path}/kogpt2-wellnesee-auto-regressive-20201022-add-chatbotdata.pth"

n_epoch = 3         # Num of Epoch
batch_size = 2      # 배치 사이즈
ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)
save_step = 100 # 학습 저장 주기
learning_rate = 2e-5  # Learning Rate

dataset= WellnessAutoRegressiveDataset(data_path)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

checkpoint = torch.load(load_ckpt_path, map_location=device)

model = DialogKoGPT2()
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

loss_fct = torch.nn.CrossEntropyLoss(ignore_index=3)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

losses =[]
for epoch in range(n_epoch):
    count = 0
    with tqdm(total=len(train_loader), desc=f"Train({epoch})") as pbar:
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            data = torch.stack(data)  # list of Tensor로 구성되어 있기 때문에 list를 stack을 통해 변환해준다.
            data = data.transpose(1, 0)
            data= data.to(ctx)

            outputs = model(data, labels=data)
            _, logits = outputs[:2]

            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = data[..., 1:].contiguous()

            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            loss.backward()
            optimizer.step()

            losses.append(loss.item())

            # if count % 10 == 0:
            #     print('epoch no.{} train no.{}  loss = {}'.format(epoch, count + 1, loss))
            if (count > 0 and count % save_step == 0) or (len(data) < batch_size):
                torch.save({
                    'epoch': epoch,
                    'train_no': count,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss
                }, save_ckpt_path)
            count += 1
            pbar.update(1)
            pbar.set_postfix_str(f"Loss: {loss.item():.3f} ({np.mean(losses):.3f})")

Train(2):  52%|█████▏    | 7258/13840 [37:36<26:27,  4.15it/s, Loss: 0.430 (1.310)]

Buffered data was truncated after reaching the output size limit.